In [1]:
# Import dependencies
from pathlib import Path
import pandas as pd
from datetime import datetime
import geopandas as gpd
from shapely.geometry import Point
import sqlite3
from sqlalchemy import create_engine
from shapely.geometry import shape, mapping, Polygon
import json
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.graph_objects as go

# Cleaning data and creating Data Base

In [2]:
# Import csv file and create df
data_path = Path('Resources/Crime_Data_from_2020_to_Present.csv')
data_df = pd.read_csv(data_path)
data_df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277


In [3]:
# Drop not needed columns
data1_df = data_df.drop(columns=['Date Rptd', 'TIME OCC','AREA', 'AREA NAME', 
                              'Rpt Dist No', 'Part 1-2', 'Crm Cd',
                              'Mocodes', 'Vict Age','Status','Vict Sex','Vict Age','Premis Cd','Premis Desc','Weapon Used Cd','Weapon Used Cd','Weapon Desc','Status','Status Desc', 'Crm Cd 1', 
                              'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 
                              'Cross Street'])

data1_df.head()

,DR_NO,DATE OCC,Crm Cd Desc,Vict Descent,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,VEHICLE - STOLEN,O,34.0375,-118.3506
1,200106753,02/08/2020 12:00:00 AM,BURGLARY FROM VEHICLE,O,34.0444,-118.2628
2,200320258,11/04/2020 12:00:00 AM,BIKE - STOLEN,X,34.0210,-118.3002
3,200907217,03/10/2020 12:00:00 AM,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),O,34.1576,-118.4387
4,220614831,08/17/2020 12:00:00 AM,THEFT OF IDENTITY,H,34.0944,-118.3277


In [4]:
# Obtain 'Year' column

# Split date and time from "DATE OCC" column
data1_df[['Date', 'Time']] = data1_df['DATE OCC'].str.split(' ', n=1, expand=True)

# Convert 'Date' to datetime using the correct format
data1_df['Date'] = pd.to_datetime(data1_df['Date'], format='%m/%d/%Y')

# Extract the year and create a new column 'Year'
data1_df['Year'] = data1_df['Date'].dt.year

# Drop not needed columns
data2_df = data1_df.drop(columns=['DATE OCC', 'Date', 'Time'])
data2_df.head()

,DR_NO,Crm Cd Desc,Vict Descent,LAT,LON,Year
0,190326475,VEHICLE - STOLEN,O,34.0375,-118.3506,2020
1,200106753,BURGLARY FROM VEHICLE,O,34.0444,-118.2628,2020
2,200320258,BIKE - STOLEN,X,34.0210,-118.3002,2020
3,200907217,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),O,34.1576,-118.4387,2020
4,220614831,THEFT OF IDENTITY,H,34.0944,-118.3277,2020


In [5]:
# Replace values in "Vict Descent' column

# Mapping dictionary to replace "Vict Descent" column
descent_mapping = {
    'A': 'Other Asian',
    'B': 'Black',
    'C': 'Chinese',
    'D': 'Cambodian',
    'F': 'Filipino',
    'G': 'Guamanian',
    'H': 'Hispanic/Latin/Mexican',
    'I': 'American Indian/Alaskan Native',
    'J': 'Japanese',
    'K': 'Korean',
    'L': 'Laotian',
    'O': 'Other',
    'P': 'Pacific Islander',
    'S': 'Samoan',
    'U': 'Hawaiian',
    'V': 'Vietnamese',
    'W': 'White',
    'X': 'Unknown',
    'Z': 'Asian Indian'
}

# Replace values of "Vict Descent" column
data2_df.loc[:, 'Vict Descent'] = data2_df['Vict Descent'].map(descent_mapping)
data2_df.head()

,DR_NO,Crm Cd Desc,Vict Descent,LAT,LON,Year
0,190326475,VEHICLE - STOLEN,Other,34.0375,-118.3506,2020
1,200106753,BURGLARY FROM VEHICLE,Other,34.0444,-118.2628,2020
2,200320258,BIKE - STOLEN,Unknown,34.0210,-118.3002,2020
3,200907217,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),Other,34.1576,-118.4387,2020
4,220614831,THEFT OF IDENTITY,Hispanic/Latin/Mexican,34.0944,-118.3277,2020


In [6]:
# Obtain "Zip Code" column using geopandas

# Import zip code geojson data
zip_codes_path = Path('Resources/LA_County_ZIP_Codes.geojson')
zip_codes_df = gpd.read_file(zip_codes_path)
crs = 'EPSG:4326'
zip_codes_df.crs = crs

# Convert latitude and longitude to point geometries
geometry = [Point(xy) for xy in zip(data2_df['LON'], data2_df['LAT'])]
data3_df = gpd.GeoDataFrame(data2_df, crs=crs, geometry=geometry)

# Spatial join to obtain zip codes
data4_df = gpd.sjoin(data3_df, zip_codes_df, how='left', predicate='intersects')

# Drop not needed columns
cleaned_data_df = data4_df.drop(columns=['LAT', 'LON', 'geometry', 'index_right', 'OBJECTID', 'Shape_Length', 'Shape_Area'])

cleaned_data_df.rename(columns={'DR_NO': 'ID', 'Crm Cd Desc': 'Crime Type', 'Vict Descent': 'Victim Descent', 'ZIPCODE': 'Zip Code'}, inplace=True)
cleaned_data_df.head()

DriverError: Resources\LA_County_ZIP_Codes.geojson: No such file or directory

In [ ]:
# Create a sqlite file
conn = sqlite3.connect('Resources/LA_crime.sqlite')
cleaned_data_df.to_sql('LA_crime_data', conn, if_exists='replace', index=False)
conn.close()

# Visualizations and Dashboard

In [ ]:
# Create Engine for data
LA_crime_path = Path('Resources/LA_crime.sqlite')
engine = create_engine(f'sqlite:///{LA_crime_path}')
conn = engine.connect()

In [ ]:
# Query all columns in the table and exclude 2024 data
LA_crime_df = pd.read_sql('SELECT * FROM LA_crime_data WHERE Year <> 2024', conn)

# Drop records with no ZIPCODE
LA_crime_df = LA_crime_df.dropna(subset=['Zip Code'])

LA_crime_df.head()

,ID,Crime Type,Victim Descent,Year,Zip Code
0,190326475,VEHICLE - STOLEN,Other,2020,90016
1,200106753,BURGLARY FROM VEHICLE,Other,2020,90015
2,200320258,BIKE - STOLEN,Unknown,2020,90007
3,200907217,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),Other,2020,91423
4,220614831,THEFT OF IDENTITY,Hispanic/Latin/Mexican,2020,90028


In [ ]:
# Close connection with Data base
conn.close()

In [ ]:
# Obtain all unique Zip Codes in data
LA_zip_codes = LA_crime_df['Zip Code'].unique()

# Create a df with all the Zip Codes
LA_zip_codes_df = pd.DataFrame({'Zip Code': LA_zip_codes})

# Load Zip Code geojson
with open('Resources/LA_County_ZIP_Codes.geojson') as file:
        zip_codes =  json.load(file)

# Create a list for the filtered features
filtered_features = []

# Filter features based on zip codes needed
for feature in zip_codes['features']:
    if feature['properties']['ZIPCODE'] in LA_zip_codes:
        filtered_features.append(feature)

# Create Dictionary with filtered features    
filtered_data = {
    'type': 'FeatureCollection',
    'features': filtered_features
}

# Loop to simplify geojson precision
for feature in filtered_data['features']:
    if feature['geometry']['type'] == 'Polygon':
        for coords in feature['geometry']['coordinates']:
            for coord in coords:
                coord[0] = round(coord[0], 4)
                coord[1] = round(coord[1], 4)
    elif feature['geometry']['type'] == 'MultiPolygon':
        for polygon in feature['geometry']['coordinates']:
            for coords in polygon:
                for coord in coords:
                    coord[0] = round(coord[0], 4)
                    coord[1] = round(coord[1], 4)

# Loop to simplify geojson geometries
for feature in filtered_data['features']:
        geometry = shape(feature['geometry'])
        simplified_geometry = geometry.simplify(0.001, preserve_topology=True)
        feature['geometry'] = mapping(simplified_geometry)

with open('Resources/zip_codes_simplified.geojson', 'w') as f:
    json.dump(filtered_data, f)

# Load zipcode geojson simplified
with open('Resources/zip_codes_simplified.geojson') as file:
        zip_codes_simplified =  json.load(file)

In [ ]:
# Create the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1('Los Angeles County Crime Data Dashboard',
            style={'font-family': 'Arial, sans-serif', 'font-weight': 'bold'}),
    html.H3('Data from 2020 to 2023',
            style={'font-family': 'Arial, sans-serif'}),
    html.Br(),
    html.H2('Select a year', style={'font-family': 'Arial, sans-serif', 'font-size': '25px'}),
    dcc.Dropdown(
        id='dropdown',
        options=['All Years', '2020', '2021', '2022', '2023'],
        value='All Years',
        clearable=False,
        style={'font-family': 'Arial, sans-serif', 'font-size': '25px', 'width': '140px', 'height':'22px'}),
    html.Br(),
    html.Div([
        html.Div([
            html.H3('bar chart')]),
        #     dcc.Graph(id='bar',
        #         style={'width': '1000px', 'height':'1000px'})]),
        html.Div([
            html.H2('Select one of the top 3 type of crimes', style={'font-family': 'Arial, sans-serif', 'font-size': '20px'}),
            dcc.RadioItems(id='radio',
                style={'font-family': 'Arial, sans-serif', 'font-size': '20px'}),
            dcc.Graph(id='map',
                style={'width': '1000px', 'height':'1000px'})]),
        ], style={'display': 'flex', 'flex-direction': 'row'}),
    html.Br(),
    html.H3('tree chart'),
    # dcc.Graph(id='tree',
    #     style={'width': '1000px', 'height':'1000px'}),
])

# Define callback to update radio options based on the selected year
@app.callback(
    Output('radio', 'options'),
    Output('radio', 'value'),
    Input('dropdown', 'value'),
)

# Function to define radio options
def update_radio(year):

    # Filter data for the year
    if year == 'All Years':
        data_year = LA_crime_df
    else:
        data_year = LA_crime_df[LA_crime_df['Year']== int(year)]

    # Create a list with the top 3 crime types of the year
    top_crime = pd.DataFrame(data_year.groupby('Crime Type')['ID'].count()).reset_index()
    top_crime_sorted = top_crime.sort_values(by='ID', ascending=False)
    top_crime_list = list(top_crime_sorted['Crime Type'].head(3))

    # Create the radio button options
    radio = [{'label': crime_type, 'value': crime_type} for crime_type in top_crime_list]

    # Define default option
    radio_default = top_crime_list[0]

    return radio, radio_default

# Define callback to create the visualizations based on year and crime type
@app.callback(
    Output('map', 'figure'),
    Input('dropdown', 'value'),
    Input('radio', 'value'),
)

# Define a function to create the visualizations
def update_map(year, radio_option):
    # Filter data for the year
    if year == 'All Years':
        data_year = LA_crime_df
    else:
        data_year = LA_crime_df[LA_crime_df['Year'] == int(year)]

    # BAR CHART
    
    

    # MAP
    # Filter the data for the current crime type
    data_crime = data_year[data_year['Crime Type'] == radio_option]
    
    # Group by Zip Code and count crime
    zip_code_crime = data_crime.groupby('Zip Code').size().reset_index(name='ID')

    # Merge the two df to obtain a list of all the zip codes with the count and replace NaN with 0
    map_df = pd.merge(LA_zip_codes_df, zip_code_crime, on='Zip Code', how='left').fillna(0)

    # Create map
    map = go.Figure(go.Choroplethmapbox(
            geojson= zip_codes_simplified,
            locations= map_df['Zip Code'],
            featureidkey= 'properties.ZIPCODE',
            z= map_df['ID'],
            colorscale= 'reds',
            colorbar_title='Crime Count'           
    ))

    map.update_layout(
        mapbox_style="carto-positron",
        mapbox_zoom=9.3,
        mapbox_center = {"lat": 34.05, "lon": -118.42},
        title={"text": f"<b>Crime level by Zip Code - {year}<br>{radio_option}</b>",
                "x": 0.5,
                "y": 0.95,
                "xanchor": "center",
                "yanchor": "top",
                "font": {"size": 20, "family": "Arial"}
        }
    )
    
    # LINE CHART
    # Dependencies 
    %matplotlib inline
    import matplotlib.pyplot as plt

    # keep two columns for a line plot
    columns = ['ID', 'Year']
    total_crime_df = LA_crime_df[columns]
    
    # Count the total crimes per year
    trend_df = total_crime_df.groupby(total_crime_df['Year']).count()
    trend_df.reset_index(inplace=True)

    # Turn data type of year into a string
    trend_df['Year']=trend_df['Year'].astype(str)

    # Create a line graph to show the crime count 
    x_axis = trend_df['Year']
    y_axis = trend_df['Record Number']
    plt.plot(x_axis, y_axis, marker="s", color="Red", linewidth=1)
    plt.xlabel('Year')
    plt.ylabel('Total Crimes')
    plt.grid()
    plt.savefig('Yi_line.png')
    plt.show()


    # TREE CHART

    return map
# bar, tree

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)